<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6f9af320e65b13bc44a7353df2178795a1fbe77f22f28599390119eac0dc462f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 13:04:53
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: 8.36 K (0.06%)
Current PnL: -27.66 L (-17.87%)
CY Booked + Current PnL: -13.37 L (-8.64%)
-------------------
Total profit:  1.42 L
Total loss:  -29.08 L
-------------------
Total Booked + Current PnL: 13.61 L (10.65%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.30 L (66.21%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,2.47,10.66,8.16,19.69,17315.0,20448.0,212190.0,-0.11,62.0,X-MC,2.54,78.0,1.18,1.53,18.19,XY25,NTT,AC
77,TTKPRESTIG,770.00,-1.17,-20.11,25.29,0.09,20360.0,-20271.0,80506.0,86.77,27.0,M-SC,2.38,253.0,-1.00,0.58,5.77,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.95,-4.45,27.74,22.05,25969.0,-4365.0,93615.0,-17.80,48.0,H-SC,6.76,164.0,-0.17,0.67,36.08,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.82,-16.43,19.69,0.02,28723.0,-28685.0,145874.0,122.66,41.0,M-SC,13.65,234.0,-1.00,1.05,26.88,OX40N,NTT,PAINTS
43,ITC,452.00,0.36,-1.66,12.65,10.78,29897.0,-3982.0,236336.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.70,3.49,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,12.12,-42.09,191.90,69.05,147884.0,-56003.0,77063.0,-53.08,64.0,H-SC,22.14,140.0,-0.38,0.56,18.85,SR,ATH,IT
6,ASIANTILES,137.00,5.19,-10.18,102.27,81.67,86169.0,-9554.0,84256.0,7577.78,62.0,L-SC,12.47,271.0,-0.11,0.61,63.17,XR,NTT,CERAMICS
82,VOLTAS,1530.00,2.47,10.66,8.16,19.69,17315.0,20448.0,212190.0,-0.11,62.0,X-MC,2.54,78.0,1.18,1.53,18.19,XY25,NTT,AC
73,TCS,4389.97,1.86,-9.88,33.93,20.71,115661.0,-37356.0,340881.0,-22.33,68.0,X-LC,4.05,1.0,-0.32,2.46,13.90,X40,ATH,IT
71,TANLA,1943.92,1.85,-36.52,246.08,119.69,422962.0,-98881.0,171880.0,-35.93,45.0,H-SC,7.82,156.0,-0.23,1.24,35.78,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.24,-63.42,173.58,0.07,87094.0,-87002.0,50175.0,1738.38,51.0,L-SC,2.35,268.0,-1.00,0.36,24.79,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.0,-2.85,-46.24,223.93,74.15,196964.0,-75647.0,87958.0,-258.45,39.0,H-SC,22.87,149.0,-0.38,0.63,12.75,XR,NTT,CHEMICALS
16,CERA,9475.0,-2.41,-30.70,78.77,23.89,96021.0,-54003.0,121900.0,-33.17,40.0,H-SC,11.06,157.0,-0.56,0.88,5.28,OX40N,NTT,CERAMICS
14,CAMPUS,393.0,-2.25,-13.26,53.67,33.28,78102.0,-22253.0,145522.0,-30.19,32.0,M-SC,2.28,222.0,-0.28,1.05,12.79,XY24,NTT,FOOTWEAR
57,RAJOOENG,143.1,-2.01,-37.08,121.89,39.61,78607.0,-38010.0,64490.0,-57.37,32.0,H-SC,29.95,135.0,-0.48,0.46,1.61,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.16,-1.82,111.77,107.91,162014.0,-2691.0,144953.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.04,8.08,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.82,-16.43,19.69,0.02,28723.0,-28685.0,145874.0,122.66,41.0,M-SC,13.65,234.0,-1.00,1.05,26.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.17,-20.11,25.29,0.09,20360.0,-20271.0,80506.0,86.77,27.0,M-SC,2.38,253.0,-1.00,0.58,5.77,OX40N,NTT,DURABLES
66,SIS,528.00,0.05,-24.18,59.93,21.26,50454.0,-26844.0,84188.0,1986.01,50.0,H-SC,4.96,166.0,-0.53,0.61,13.75,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.03,-25.91,53.15,13.47,106194.0,-69867.0,199800.0,-69.48,32.0,H-SC,6.04,158.0,-0.66,1.44,1.59,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.81,7.98,59.54,72.27,108938.0,13518.0,182966.0,-7.44,68.0,M-LC,2.8,99.0,0.12,1.32,14.65,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.81,7.98,59.54,72.27,108938.0,13518.0,182966.0,-7.44,68.0,M-LC,2.80,99.0,0.12,1.32,14.65,XR,NTT,IT
25,FINCABLES,1641.55,1.16,-1.82,111.77,107.91,162014.0,-2691.0,144953.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.04,8.08,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.50,-3.39,113.99,106.72,135818.0,-4187.0,119149.0,-52.15,39.0,H-SC,3.43,139.0,-0.03,0.86,19.19,AR,ATH,MISC
37,IEX,219.00,0.83,-4.04,54.89,48.63,105315.0,-8088.0,191866.0,-36.24,49.0,H-SC,13.53,136.0,-0.08,1.38,7.04,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.29,-3.47,40.50,35.63,81933.0,-7271.0,202303.0,-17.18,37.0,H-MC,2.77,119.0,-0.09,1.46,12.62,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.35,-18.69,91.21,55.47,180425.0,-45473.0,197813.0,-28.03,25.0,X-MC,13.93,64.0,-0.25,1.43,1.36,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.39,-10.84,43.69,28.11,78456.0,-21835.0,179575.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.12,-24.44,103.14,53.50,235165.0,-73745.0,228006.0,-63.37,27.0,X-MC,3.79,58.0,-0.31,1.64,1.27,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.03,-39.43,123.20,35.20,96023.0,-50729.0,77941.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.27,-29.21,73.71,22.98,117877.0,-65978.0,159920.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.58,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,1.54,-2.37,27.50,24.48,67618.0,-5973.0,245883.0,-81.32,42.0,X-SC,0.59,86.0,-0.09,1.77,20.22,X40N,NTT,MISC
12,BERGEPAINT,680.0,-1.17,-5.12,27.68,21.14,59707.0,-11636.0,215703.0,-11.79,36.0,X-MC,1.69,74.0,-0.19,1.55,21.58,XY24,NTT,PAINTS
20,DABUR,735.0,-0.23,-3.61,49.09,43.70,117377.0,-8963.0,239105.0,-11.62,34.0,X-MC,1.98,72.0,-0.08,1.72,10.89,XY24,BTT,FMCG
43,ITC,452.0,0.36,-1.66,12.65,10.78,29897.0,-3982.0,236336.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.70,3.49,X40,NTT,FMCG
78,UNITDSPR,1644.0,-1.91,2.67,17.62,20.77,42359.0,6260.0,240404.0,-6.25,41.0,X-MC,2.37,62.0,0.15,1.73,9.78,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.19,-26.22,122.50,64.15,215049.0,-62401.0,175550.0,-56.75,31.0,X-SC,3.01,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.03,-39.43,123.20,35.20,96023.0,-50729.0,77941.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-0.39,-10.84,43.69,28.11,78456.0,-21835.0,179575.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,0.11,-20.66,78.35,41.51,163720.0,-54405.0,208960.0,-5.42,35.0,X-MC,7.50,61.0,-0.33,1.51,0.11,XY25,ATH,FMCG
75,TMPV,600.00,-0.27,-29.21,73.71,22.98,117877.0,-65978.0,159920.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.58,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.24,-31.10,107.30,42.84,48062.0,-20214.0,44792.0,-54.82,38.0,X-SC,35.18,83.0,-0.42,0.32,1.34,XY24,NTT,MISC
59,RELAXO,1176.00,-0.19,-48.13,193.63,52.31,145792.0,-69866.0,75294.0,-44.73,40.0,X-SC,7.25,91.0,-0.48,0.54,0.79,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.03,-39.43,123.20,35.20,96023.0,-50729.0,77941.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.14,-5.67,31.54,24.08,37769.0,-7202.0,119748.0,-13.35,31.0,X-SC,5.40,89.0,-0.19,0.86,12.83,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.75,-17.52,73.76,43.32,99089.0,-28532.0,134340.0,-29.17,33.0,X-SC,5.03,90.0,-0.29,0.97,3.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.86,-9.88,33.93,20.71,115661.0,-37356.0,340881.0,-22.33,68.0,X-LC,4.05,1.0,-0.32,2.46,13.90,X40,ATH,IT
41,INFY,1972.00,1.49,10.68,21.29,34.24,73726.0,33407.0,346295.0,-13.85,66.0,X-LC,4.57,2.0,0.45,2.50,19.84,X40,NTT,IT
75,TMPV,600.00,-0.27,-29.21,73.71,22.98,117877.0,-65978.0,159920.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.58,XY24,NTT,AUTO
81,VBL,671.64,0.18,-4.72,42.39,35.67,127571.0,-14897.0,300945.0,-16.08,51.0,X-LC,4.05,4.0,-0.12,2.17,8.48,X40N,ATH,FMCG
43,ITC,452.00,0.36,-1.66,12.65,10.78,29897.0,-3982.0,236336.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.70,3.49,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.59,-38.08,114.61,32.89,54794.0,-29398.0,47809.0,-690.25,51.0,L-MC,6.25,259.0,-0.54,0.34,31.66,XR,NTT,BANKS
6,ASIANTILES,137.00,5.19,-10.18,102.27,81.67,86169.0,-9554.0,84256.0,7577.78,62.0,L-SC,12.47,271.0,-0.11,0.61,63.17,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.95,-4.45,27.74,22.05,25969.0,-4365.0,93615.0,-17.80,48.0,H-SC,6.76,164.0,-0.17,0.67,36.08,XR,ATH,FINANCE
13,BSOFT,831.70,-0.20,-21.86,92.97,50.79,102179.0,-30748.0,109905.0,0.12,64.0,H-SC,6.58,151.0,-0.30,0.79,27.97,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.82,-16.43,19.69,0.02,28723.0,-28685.0,145874.0,122.66,41.0,M-SC,13.65,234.0,-1.00,1.05,26.88,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.20,-21.86,92.97,50.79,102179.0,-30748.0,109905.0,0.12,64.0,H-SC,6.58,151.0,-0.30,0.79,27.97,XR,ATH,IT
61,ROUTE,2227.21,12.12,-42.09,191.90,69.05,147884.0,-56003.0,77063.0,-53.08,64.0,H-SC,22.14,140.0,-0.38,0.56,18.85,SR,ATH,IT
41,INFY,1972.00,1.49,10.68,21.29,34.24,73726.0,33407.0,346295.0,-13.85,66.0,X-LC,4.57,2.0,0.45,2.50,19.84,X40,NTT,IT
84,WIPRO,420.00,0.81,7.98,59.54,72.27,108938.0,13518.0,182966.0,-7.44,68.0,M-LC,2.80,99.0,0.12,1.32,14.65,XR,NTT,IT
73,TCS,4389.97,1.86,-9.88,33.93,20.71,115661.0,-37356.0,340881.0,-22.33,68.0,X-LC,4.05,1.0,-0.32,2.46,13.90,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,44.92
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.75
MC    30.93
LC    24.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.57
X40      23.17
X40N     13.00
XR        9.77
XY25      9.15
AR        8.99
OX40N     7.51
SR        1.05
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.44
H-SC    23.74
X-LC    20.83
M-SC    11.56
X-SC     8.01
H-MC     4.81
L-SC     1.44
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.34,-8.35,43.19
IT,13.24,-14.54,72.13
FINANCE,9.44,-21.71,74.64
MISC,7.28,-28.41,80.94
ELECTRICAL,5.99,-13.03,54.43
PAINTS,5.80,-14.72,31.76
INSURANCE,4.81,-1.65,36.44
PHARMA,4.14,-2.45,34.70
AUTO,2.80,-34.67,81.36


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3235810.0,21
AR,1324281.0,10
XR,1304203.0,13
X40,1008417.0,14
X40N,880730.0,9
OX40N,748848.0,10
XY25,373096.0,6
SR,275971.0,2
MH,78161.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3684064.0,25
M-SC,1452892.0,15
X-MC,1434082.0,16
X-LC,837437.0,11
X-SC,806536.0,8
H-MC,412914.0,3
L-SC,262160.0,3
M-LC,108938.0,1
H-LC,79572.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295051.0      6
           AR         900879.0      5
M-SC       XY24       832242.0      6
H-SC       XR         788743.0      7
X-MC       X40        494056.0      7
           XY24       412249.0      3
           X40N       346742.0      4
H-SC       OX40N      345259.0      4
X-LC       X40        321204.0      5
M-SC       OX40N      316495.0      5
X-SC       X40N       312499.0      3
           XY24       300880.0      3
H-SC       SR         275971.0      2
X-LC       X40N       221489.0      2
H-MC       AR         216337.0      2
X-LC       XY24       198811.0      2
H-MC       XY24       196577.0      1
X-SC       X40        193157.0      2
X-MC       XY25       181035.0      2
M-SC       XR         176662.0      2
L-SC       XR         175066.0      2
M-SC       AR         127493.0      2
M-LC       XR         108938.0      1
X-LC       XY25        95933.0      2
L-SC       OX40N       87094.0      1
H-LC       AR          79572.0      1
H-SC       MH          78161.0      1
M-MC       XY25        59395.0      1
L-MC       XR          54794.0      1
L-LC       XY25        36733.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
